In [1]:
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from glob import glob
import os

In [2]:
LABEL_INT_DICT = np.unique(pd.read_csv('ucf11_train_vid.csv')['label'])
LABEL_STR_DICT = {k:v for v, k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_INT_DICT)
pprint(LABEL_STR_DICT)

array(['biking', 'diving', 'golf', 'juggle', 'jumping', 'riding',
       'shooting', 'spiking', 'swing', 'tennis', 'walk'], dtype=object)
{'biking': 0,
 'diving': 1,
 'golf': 2,
 'juggle': 3,
 'jumping': 4,
 'riding': 5,
 'shooting': 6,
 'spiking': 7,
 'swing': 8,
 'tennis': 9,
 'walk': 10}


In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

MAX_FRAMES = 10
NUM_FEATURES = 1280

def build_model():
    inputs = keras.Input((MAX_FRAMES, NUM_FEATURES))
    x = keras.layers.LSTM(64, return_sequences=True)(
        inputs
    )
    x = keras.layers.LSTM(64, return_sequences=False)(
        x
    )
    x = keras.layers.Dropout(0.3)(x)
    outputs = keras.layers.Dense(len(LABEL_INT_DICT), activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    return model

adam = keras.optimizers.Adam(lr=0.0001)
model = build_model()

model.compile(
    optimizer=adam,
    loss = 'sparse_categorical_crossentropy',
    metrics='accuracy'
)

model.summary()

C:\Users\khj_a\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 1280)]        0         
                                                                 
 lstm (LSTM)                 (None, 10, 64)            344320    
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 11)                715       
                                                                 
Total params: 378,059
Trainable params: 378,059
Non-trainable params: 0
_________________________________________________________________


In [4]:
import albumentations as A
import cv2
import math
import tensorflow as tf
from tensorflow import keras

class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size,
        mode='train', shuffle=True):
        assert mode in ['train', 'valid']
        self.mode = mode
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.npy_paths = glob(
            f'UCF11_updated_npy/{mode}/*.npy'
        )
    
    def __len__(self):
        return math.ceil(len(self.npy_paths) / self.batch_size)
    
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.npy_paths[strt:fin]
    
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
    
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for npy_path in data:
            npy = np.load(npy_path)
            
            label = npy_path.split('\\')[-1].split('_')[0]
            label = LABEL_STR_DICT[label]
            
            batch_x.append(npy)
            batch_y.append(label)
            
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.npy_paths)
    
train_generator = DataGenerator(
    mode='train',
    batch_size=128,
    shuffle=True)

valid_generator = DataGenerator(
    mode='valid',
    batch_size=128,
    shuffle=True)

In [5]:
for x, y in train_generator:
    print(x.shape)
    print(y.shape)
    break

(128,)
(128,)


C:\Users\khj_a\AppData\Local\Temp/ipykernel_15000/426474324.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(batch_x), np.array(batch_y)


In [14]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    verbose=1
)

Epoch 1/30
2/2 [==============================] - 3s 47ms/step - loss: 2.3971 - accuracy: 0.1273
Epoch 2/30
2/2 [==============================] - 0s 111ms/step - loss: 2.3426 - accuracy: 0.1636
Epoch 3/30
2/2 [==============================] - 0s 114ms/step - loss: 2.2789 - accuracy: 0.2045
Epoch 4/30
2/2 [==============================] - 0s 112ms/step - loss: 2.2448 - accuracy: 0.2500
Epoch 5/30
2/2 [==============================] - 0s 105ms/step - loss: 2.1873 - accuracy: 0.3136
Epoch 6/30
2/2 [==============================] - 0s 57ms/step - loss: 2.1524 - accuracy: 0.3500
Epoch 7/30
2/2 [==============================] - 0s 102ms/step - loss: 2.1095 - accuracy: 0.4273
Epoch 8/30
2/2 [==============================] - 0s 106ms/step - loss: 2.0430 - accuracy: 0.4727
Epoch 9/30
2/2 [==============================] - 0s 107ms/step - loss: 1.9872 - accuracy: 0.5545
Epoch 10/30
2/2 [==============================] - 0s 105ms/step - loss: 1.9571 - accuracy: 0.5864
Epoch 11/30
2/2 [====